In [13]:
from icrawler.builtin import GoogleImageCrawler
import datetime
def datetime2tuple(date):
    return (date.year, date.month, date.day)
def gather_data(n_total_images,n_per_crawl,keyword, folder_name):
    delta = datetime.timedelta(days=30)
    end_day = datetime.datetime(2023, 3, 17)
    for i in range(int(n_total_images / n_per_crawl )):
        start_day = end_day - delta
        google_crawler = GoogleImageCrawler(downloader_threads=4, storage={'root_dir': folder_name})
        google_crawler.crawl(keyword=keyword[i], filters={'date':(datetime2tuple(start_day), datetime2tuple(end_day))}, file_idx_offset=i*n_per_crawl , max_num=n_per_crawl)
        end_day = start_day - datetime.timedelta(days=1)

gather_data(700,100,['packages ready to ship','packages about to be sent','cardboard boxes','packages','box packages','sealed packages','delivery cardboard package box'],'goodpackages')
gather_data(200,100,['ripped cardboard boxes','ripped packages'],'rippedpackages') #ripped
gather_data(200,100,['destroyed packages','damaged cardboard boxes'],'crushedpackages') #crushed
gather_data(200,100,['cardboard boxes with water damage','wet cardboard boxes'],'leakingpackages')#leaking
gather_data(200,100,['burned cardboard','charred cardboard'],'burnedpackages') # burned
gather_data(200,100,['cardboard recycling pile','broken down cardboard boxes'],'foldedpackages')#folded

2023-03-18 21:50:34,902 - INFO - icrawler.crawler - start crawling...
2023-03-18 21:50:34,903 - INFO - icrawler.crawler - starting 1 feeder threads...
2023-03-18 21:50:34,904 - INFO - feeder - thread feeder-001 exit
2023-03-18 21:50:34,905 - INFO - icrawler.crawler - starting 1 parser threads...
2023-03-18 21:50:34,908 - INFO - icrawler.crawler - starting 4 downloader threads...
2023-03-18 21:50:35,631 - INFO - parser - parsing result page https://www.google.com/search?q=packages+ready+to+ship&ijn=0&start=0&tbs=cdr%3A1%2Ccd_min%3A02%2F15%2F2023%2Ccd_max%3A03%2F17%2F2023&tbm=isch
2023-03-18 21:50:35,787 - ERROR - downloader - Response status code 403, file https://external-preview.redd.it/ypLY89Swo8viOjhaaNSAbltbRjXZQQA8he533rTAj24.jpg
2023-03-18 21:50:35,799 - INFO - downloader - image #1	https://dtpmhvbsmffsz.cloudfront.net/posts/2013/03/16/5144dfbeac4d245b1c01b73a/m_5144dfc3ac4d245b1c01b741.jpg
2023-03-18 21:50:35,803 - INFO - downloader - image #2	https://dtpmhvbsmffsz.cloudfront.ne

2023-03-18 21:55:18,189 - INFO - downloader - downloader-003 is waiting for new download tasks
2023-03-18 21:55:18,408 - INFO - downloader - downloader-001 is waiting for new download tasks
2023-03-18 21:55:18,717 - INFO - downloader - downloader-004 is waiting for new download tasks
2023-03-18 21:55:20,331 - INFO - downloader - downloader-002 is waiting for new download tasks
2023-03-18 21:55:23,192 - INFO - downloader - downloader-003 is waiting for new download tasks
2023-03-18 21:55:23,409 - INFO - downloader - downloader-001 is waiting for new download tasks
2023-03-18 21:55:23,718 - INFO - downloader - downloader-004 is waiting for new download tasks
2023-03-18 21:55:25,332 - INFO - downloader - downloader-002 is waiting for new download tasks
2023-03-18 21:55:28,199 - INFO - downloader - downloader-003 is waiting for new download tasks
2023-03-18 21:55:28,416 - INFO - downloader - downloader-001 is waiting for new download tasks
2023-03-18 21:55:28,727 - INFO - downloader - down